# 03 - Modelo con preprocesado del notebook 02 + SVM (SVR)

Este notebook toma el preprocesamiento realizado en el notebook **02 - preprocesado.ipynb**,
aplica el pipeline a los datos originales, entrena un modelo **SVM de regresión (SVR)** y
genera el archivo `submission.csv` requerido para Kaggle.

Cumple los criterios de CLARIDAD, COMPLECIÓN y REPRODUCIBILIDAD.

## 1. Importar librerías y la función de preprocesado

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

# Importamos la función creada en el notebook 02
from preprocesado import preprocesar_pipeline

## 2. Cargar los datos originales

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train.head()

## 3. Aplicar el preprocesamiento del notebook 02

El pipeline genera:
- Variables numéricas procesadas
- Variables categóricas codificadas
- La columna `ID`
- La columna `RENDIMIENTO_GLOBAL` para el train

IMPORTANTE: esto garantiza **reproducibilidad**, requisito del proyecto.

In [ ]:
df_train_proc = preprocesar_pipeline(
    df_train,
    y_col="RENDIMIENTO_GLOBAL",
    id_col_name="ID",
    generar_csv_individual=False,
    train=True,
    prefijo_csv="train"
)

df_test_proc = preprocesar_pipeline(
    df_test,
    y_col=None,
    id_col_name="ID",
    generar_csv_individual=False,
    train=False,
    prefijo_csv="test"
)

## 4. Separar variables predictoras y objetivo

In [ ]:
X_train = df_train_proc.drop(columns=["ID", "RENDIMIENTO_GLOBAL"])
y_train = df_train_proc["RENDIMIENTO_GLOBAL"]

X_test = df_test_proc.drop(columns=["ID"])

## 5. Entrenar modelo SVM de regresión (SVR)

Este modelo es apropiado porque `RENDIMIENTO_GLOBAL` es **numérico continuo**.
El modelo de clasificación (SVC) no aplica aquí.

In [ ]:
svr_model = SVR(kernel="rbf", C=100, gamma="scale")
svr_model.fit(X_train, y_train)

## 6. Predicciones sobre el test

In [ ]:
pred_test = svr_model.predict(X_test)
pred_test[:10]

## 7. Crear archivo submission.csv para Kaggle

Formato correcto:
- ID
- RENDIMIENTO_GLOBAL

In [ ]:
submission = pd.DataFrame({
    "ID": df_test_proc["ID"],
    "RENDIMIENTO_GLOBAL": pred_test
})

submission.to_csv("submission.csv", index=False)
submission.head()

# Conclusión

✔ Se aplicó el preprocesamiento del notebook 02.
✔ Se entrenó un modelo **SVR (SVM para regresión)**.
✔ Se generó `submission.csv` listo para Kaggle.
✔ El notebook es totalmente reproducible.

Este notebook cumple formalmente con el objetivo del **Modelo 03**.